In [ ]:
import torch 
from torchvision.models import resnet50,ResNet50_Weights,resnet152,ResNet152_Weights
import torchvision
import os
os.environ['TORCH_HOME']=r"D:\torch_model"
projectPath=r'D:\project_meta\NNproject\NNI'
import os
import sys
root=os.path.abspath(projectPath)
print(root)
sys.path.append(root)
os.environ["TORCH_HOME"]=r"E:\Data\torch-model"
import warnings  
warnings.filterwarnings("ignore", category=RuntimeWarning, module="mne")



In [ ]:
from project.trainer.ClassfierTrainer import ClassfierTrainer
from torch.utils.data import DataLoader
import torchvision
from project.dataprocess.transform import AddSaltPepperNoise

weights = torchvision.models.ResNet152_Weights.IMAGENET1K_V1
net = torchvision.models.resnet152(weights=weights)

In [ ]:
train_transforms = torchvision.transforms.Compose([
    AddSaltPepperNoise(0.2),
    torchvision.transforms.transforms.RandomRotation(10),  # 随机旋转

    torchvision.transforms.transforms.ColorJitter(0.4, 0.4, 0.4, 0.2),  # 随机颜色抖动
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5]),
])


train_dataset=torchvision.datasets.ImageFolder(root=r'D:\project_meta\NNproject\NNI\output\video_frames\IDSplite\train',transform=train_transforms)
train_dataloader=DataLoader(train_dataset,batch_size=64,shuffle=True)

test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5]),
])
test_dataset=torchvision.datasets.ImageFolder(root=r'D:\project_meta\NNproject\NNI\output\video_frames\IDSplite\test',transform=test_transforms)

test_dataloader=DataLoader(test_dataset,batch_size=128,shuffle=True)


In [ ]:
from project.trainer.ClassfierTrainer import ClassfierTrainer
optim=torch.optim.SGD(net.parameters(),lr=1,weight_decay=5e-4)
lr_sch=torch.optim.lr_scheduler.MultiStepLR(optim,[1,2,3],0.7)
loss=torch.nn.CrossEntropyLoss().to(device="cuda")
classfierTrainer=ClassfierTrainer(net,epochNum=50,optimizer=optim,lossFunction=loss)

In [ ]:
classfierTrainer.TrainEpochs(epochNum=50,net=net,traindataloader=train_dataloader,testdataloader=test_dataloader,optimizer=optim,lossFunction=loss)